In [214]:
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [215]:
train_tokens = pd.read_csv('../data/train_x.csv')
train_tags = pd.read_csv('../data/train_y.csv')

In [216]:
train_data = train_tokens.join(train_tags, on="id", how = "inner", rsuffix = "_tag").drop("id_tag",axis=1)

In [217]:
sentences = []
tags_list = []
temp_tokens = []
temp_tags = []
for row in train_data.itertuples():
    word = row[2]
    tag = row[3]
    if word!='-DOCSTART-' and word!='.':
        temp_tokens.append(word)
        temp_tags.append(tag)
    if word=='.':
        sentences.append(' '.join(temp_tokens) + ' .')
        tags_list.append(' '.join(temp_tags) + ' .')
        temp_tokens = []
        temp_tags = []
    

In [218]:
def nGramTagger(n):
    dic = {}
    for line in tags:
        line = line.split(' ')
        line = ['*']*n + line
        for i in range(n,len(line)):
            if n==1:
                item = line[i]
            else:
                item = tuple(line[i-n:i])
            if item in dic:
                dic[item]+=1
            else:
                dic[item]=1
    return dic

def wordTagger():
    dic = defaultdict(int) 
    for line1,line2 in zip(sentences,tags):
        for word,tag in zip(line1.split(' '),line2.split(' ')):
            dic[(word,tag)]+=1
    return dic

unigram = nGramTagger(1)
bigram = nGramTagger(2)
trigram = nGramTagger(3)

wordtag = wordTagger()

In [219]:
def get_q(tag_penult,tag_last,tag_current):
    return float(trigram[(tag_penult, tag_last, tag_current)])/float(bigram[(tag_penult, tag_last)])

def get_e(word,tag):
    return float(wordtag[(word,tag)])/float(unigram[tuple([tag])])

In [220]:
def prob_distibution(k):
    prob = []
    tag_list = zip(*unigram.keys())[0]
    for key1 in tag_list :
        for key2 in tag_list:
            if (key1,key2) in bigram:
                num = bigram[(key1,key2)] + k
            else:
                num=k
            den = unigram[(key1,)] + (k*len(tag_list))
            prob.append(float(num)/den)
    print prob.count(0.0)
    plt.plot(sorted(prob,reverse=True))
    plt.show()

In [221]:
train = zip(sentences,tags_list)